<a href="https://colab.research.google.com/github/astrovishalthakur/MachineLearning/blob/main/Ensemble/Boosting/XGBoost/XGBoostClassifierDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
! mkdir ~/.kaggle

In [4]:
! cp "/content/drive/MyDrive/kaggle/kaggle.json" ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 21.3MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.74MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 27.8MB/s]


In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("train.csv")

In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
df.shape

(891, 12)

In [10]:
df.drop(columns=["Name", "PassengerId", "Cabin", "Ticket"], inplace=True)

In [11]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [12]:
X.head(
    
)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [13]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [14]:
# fill missing values in age and embarked
# OneHotEncoding in sex/embarked
# train the model

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [17]:
trf1 = ColumnTransformer([
    ("imputation", SimpleImputer(), [2]), # 2 here is index of age column
    ("imputation2", SimpleImputer(strategy="most_frequent"), [6])
],
remainder="passthrough")

In [18]:
from sklearn.preprocessing import OneHotEncoder

In [19]:
trf2 = ColumnTransformer([
    ("ohe", OneHotEncoder(sparse=False, handle_unknown="ignore"), [1,6])
])

In [20]:
from xgboost import XGBClassifier
clf = XGBClassifier(eta=0.01, reg_lambda=10)

In [21]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("step1",trf1),
    ("step2", trf2),
    ("step3", clf)
])

In [23]:
import warnings
warnings.filterwarnings("ignore")

In [24]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputation', SimpleImputer(),
                                                  [2]),
                                                 ('imputation2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('step2',
                 ColumnTransformer(transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('step3', XGBClassifier(eta=0.01, reg_lambda=10))])

In [25]:
y_pred = pipe.predict(X_test)

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6312849162011173

# This is Important.

### We know how to apply  gridsearch on models.

### Here we will see how to apply it on model if it is already integrated with pipeline

In [29]:
params = {}
params["step3__eta"] = [0.01, 0.05, 0.1, 0.3]
params["step3__gamma"]= [1, 10, 20, 50]
params["step3__reg_lambda"] = [1, 5, 10, 50, 100]

In [30]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring="accuracy", n_jobs=-1)
grid.fit(X, y)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('step1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('imputation',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('imputation2',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('step2',
                                        ColumnTransformer(transformers=[('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                    

In [33]:
grid.best_estimator_

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputation', SimpleImputer(),
                                                  [2]),
                                                 ('imputation2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('step2',
                 ColumnTransformer(transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('step3', XGBClassifier(eta=0.01, gamma=1))])

In [34]:
grid.best_score_

0.656619170171364

In [35]:
grid.best_params_

{'step3__eta': 0.01, 'step3__gamma': 1, 'step3__reg_lambda': 1}